# Projeto 2 - Ciência dos Dados

Nome: _____

Nome: _____

Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
# Classificador automático de sentimento


## Preparando o ambiente no jupyter:

In [32]:
%%capture

#Instalando o tweepy
!pip install tweepy
!pip install emoji
import pandas as pd
import re
from emoji import UNICODE_EMOJI
import emoji

In [21]:
import tweepy
import math
import os.path
import pandas as pd
import json
from random import shuffle
import numpy as np

___
## Autenticando no  Twitter

* Conta: ***[Preencha aqui o id da sua conta. Ex: @fulano ]***

In [15]:
#Dados de autenticação do twitter:

#Coloque aqui o identificador da conta no twitter: @fulano

#leitura do arquivo no formato JSON
with open('auth.pass') as fp:    
    data = json.load(fp)

#Configurando a biblioteca. Não modificar
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])

___
## Etapas do projeto:

### Escolha de um produto e coleta das mensagens


In [16]:
#Produto escolhido:
produto = 'Windows'

#Quantidade mínima de mensagens capturadas:
n = 500
#Quantidade mínima de mensagens para a base de treinamento:
t = 300

#Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'en'

Capturando os dados do twitter:

In [19]:
#Cria um objeto para a captura
api = tweepy.API(auth)

#Inicia a captura, para mais detalhes: ver a documentação do tweepy
i = 1
msgs = []
for msg in tweepy.Cursor(api.search, q = produto, lang=lang, tweet_mode="extended").items():
    if msg.full_text.lower()[0] != 'r' and msg.full_text.lower()[1] != 't':
        msgs.append(msg.full_text.lower())
        i += 1
        if i > n:
            break

#Embaralhando as mensagens para reduzir um possível viés
shuffle(msgs)

Salvando os dados em uma planilha Excel:

In [20]:
#Verifica se o arquivo não existe para não substituir um conjunto pronto
if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
    writer = pd.ExcelWriter('{0}.xlsx'.format(produto))

    #divide o conjunto de mensagens em duas planilhas
    dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])})
    dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

    dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])})
    dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
    writer.save()

___
### Montando o Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [92]:
Doc = pd.read_excel('Windows.xlsx', sheet_name = 0)
Doc.head()

,Treinamento,Relevancia
0,@evelynchartres oh that's good to know! i have...,1
1,how to resolve a usb security key is not worki...,1
2,"kumo ind. llc korea\naluminum &amp; u-pvc, win...",0
3,full time: heritage windows 2 - 1 braishfield ...,0
4,@sagaciouszu @aciouvistas @gorydetail it doesn...,1


In [94]:
limp=[]
for i in Doc.Treinamento:
    a=re.sub('@','',i)
    a=re.sub(':','',a)
    a=re.sub("'",'',a)
    a=re.sub('-','',a)
    a=re.sub('#','',a)
    a=re.sub('"','',a)
    a=re.sub('!','',a)
    a=re.sub(';','',a)
    a=re.sub('\n',' ',a)
    a=re.sub('\t',' ',a)
    a=re.sub('&',' ',a)
    a=a.strip('()')
    a=a.replace('.','')
    a=a.replace('?','')
    a=a.replace(',','')
    limp.append(a)
print(limp)

['evelynchartres oh thats good to know i have windows 6 haha i really need a new computer', 'how to resolve a usb security key is not working on windows 10  https//tco/haugaqmie0 https//tco/gzja1nk7ug', 'kumo ind llc korea aluminum  amp upvc windows doors modern kitchen trading  showroom sheikh mohammed bin salem road ras al khaimah uae from saturday  thursday 0900am100pm then 400pm1000pm  factory umm al quwain uae from saturday  thursday 700am  700pm https//tco/kplxyscls4', 'full time heritage windows 2  1 braishfield football club herbra pitchero https//tco/zl7ulrkfe8 https//tco/ffqw0sgspv', 'sagaciouszu aciouvistas gorydetail it doesnt function well with windows core stuff like the font will get squished but in my prototypes i use it and it makes reading my games much easier', 'blakegc windows xp', 'the f7 has been cleared and is open for traffic in both directions motorists are still advised to keep windows rolled up while driving the pass however', 'maggieelmasry h will look in al

In [95]:
Doc_limpo = pd.DataFrame()
Doc_limpo['Treinamento']=limp
Doc_limpo['Relevancia']=Doc.Relevancia
Doc_limpo.head()

,Treinamento,Relevancia
0,evelynchartres oh thats good to know i have wi...,1
1,how to resolve a usb security key is not worki...,1
2,kumo ind llc korea aluminum amp upvc windows ...,0
3,full time heritage windows 2 1 braishfield fo...,0
4,sagaciouszu aciouvistas gorydetail it doesnt f...,1


In [96]:
def is_emoji(s):
    return s in UNICODE_EMOJI

def add_space(text):
    return ''.join(' ' + char if is_emoji(char) else char for char in text).strip()

limp_final=[add_space(text) for text in limp]

print(limp_final)

['evelynchartres oh thats good to know i have windows 6 haha i really need a new computer', 'how to resolve a usb security key is not working on windows 10  https//tco/haugaqmie0 https//tco/gzja1nk7ug', 'kumo ind llc korea aluminum  amp upvc windows doors modern kitchen trading  showroom sheikh mohammed bin salem road ras al khaimah uae from saturday  thursday 0900am100pm then 400pm1000pm  factory umm al quwain uae from saturday  thursday 700am  700pm https//tco/kplxyscls4', 'full time heritage windows 2  1 braishfield football club herbra pitchero https//tco/zl7ulrkfe8 https//tco/ffqw0sgspv', 'sagaciouszu aciouvistas gorydetail it doesnt function well with windows core stuff like the font will get squished but in my prototypes i use it and it makes reading my games much easier', 'blakegc windows xp', 'the f7 has been cleared and is open for traffic in both directions motorists are still advised to keep windows rolled up while driving the pass however', 'maggieelmasry h will look in al

In [97]:
Doc_limpo = pd.DataFrame()
Doc_limpo['Treinamento']=limp_final
Doc_limpo['Relevancia']=Doc.Relevancia

In [98]:
Doc_limpo.Treinamento=Doc_limpo.Treinamento.str.replace('  ', ' ')
Doc_limpo.head()

,Treinamento,Relevancia
0,evelynchartres oh thats good to know i have wi...,1
1,how to resolve a usb security key is not worki...,1
2,kumo ind llc korea aluminum amp upvc windows d...,0
3,full time heritage windows 2 1 braishfield foo...,0
4,sagaciouszu aciouvistas gorydetail it doesnt f...,1


In [99]:
Doc_0=Doc_limpo[(Doc_limpo.Relevancia==0)]
Doc_1=Doc_limpo[(Doc_limpo.Relevancia==1)]

In [160]:
string0=" ".join(str(x) for x in Doc_0.Treinamento)
serie_0 = pd.Series(string0.split())
tabela_0=serie_0.value_counts(True).to_frame('Frequency')
tabela_0['Palavra']=tabela_0.index
tabela_0.head()

,Frequency,Palavra
the,0.037128,the
windows,0.035662,windows
to,0.020762,to
a,0.019541,a
and,0.019052,and


In [161]:
string1=" ".join(str(x) for x in Doc_1.Treinamento)
serie_1 = pd.Series(string1.split())
tabela_1=serie_1.value_counts(True).to_frame('Frequency')
tabela_1['Palavra']=tabela_1.index
tabela_1.head()

,Frequency,Palavra
windows,0.045016,windows
to,0.033031,to
a,0.025431,a
the,0.022508,the
and,0.020170,and


In [162]:
Doc = pd.read_excel('Windows.xlsx', sheet_name = 1)
Doc.head()

,Teste,Relevancia
0,bluekeep is a remote windows kernel use-after-...,1
1,"kumo ind. llc korea\naluminum &amp; u-pvc, win...",0
2,"anybody can spot me ""mcsa guide to identity wi...",1
3,thinking about moving to a @windows 10 device?...,1
4,@wocharlog windows wale raped,0


In [165]:
limp=[]
for i in Doc.Teste:
    a=re.sub('@','',i)
    a=re.sub(':','',a)
    a=re.sub("'",'',a)
    a=re.sub('-','',a)
    a=re.sub('#','',a)
    a=re.sub('"','',a)
    a=re.sub('!','',a)
    a=re.sub(';','',a)
    a=re.sub('\n',' ',a)
    a=re.sub('\t',' ',a)
    a=re.sub('&',' ',a)
    a=a.strip('()')
    a=a.replace('.','')
    a=a.replace('?','')
    a=a.replace(',','')
    limp.append(a)

In [166]:
Doc_limpo = pd.DataFrame()
Doc_limpo['Teste']=limp
Doc_limpo['Relevancia']=Doc.Relevancia
Doc_limpo.head()

,Teste,Relevancia
0,bluekeep is a remote windows kernel useafterfr...,1
1,kumo ind llc korea aluminum amp upvc windows ...,0
2,anybody can spot me mcsa guide to identity wit...,1
3,thinking about moving to a windows 10 device g...,1
4,wocharlog windows wale raped,0


In [167]:
def is_emoji(s):
    return s in UNICODE_EMOJI

def add_space(text):
    return ''.join(' ' + char if is_emoji(char) else char for char in text).strip()

limp_final=[add_space(text) for text in limp]

In [168]:
Doc_limpo = pd.DataFrame()
Doc_limpo['Teste']=limp_final
Doc_limpo['Relevancia']=Doc.Relevancia

Doc_limpo.Teste=Doc_limpo.Teste.str.replace('  ', ' ')
Doc_limpo.head()

,Teste,Relevancia
0,bluekeep is a remote windows kernel useafterfr...,1
1,kumo ind llc korea aluminum amp upvc windows d...,0
2,anybody can spot me mcsa guide to identity wit...,1
3,thinking about moving to a windows 10 device g...,1
4,wocharlog windows wale raped,0


In [ ]:
for a in Doc_limpo.Teste:
    zero=0
    um=0
    lista_pal=a.split(' ')
    for palavra in lista_pal:
        for palavra in tabela_0.Palavra:
            zero*=tabela_0.Frequency

print(zero)
        
    

___
### Verificando a performance

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que periodicamente realiza análise de sentimento e visualiza estes dados

# Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**